In [76]:
from init import *
code = r'''
#include <eosio/print.hpp>
#include <eosio/name.hpp>
#include <eosio/action.hpp>
#include <eosio/multi_index.hpp>

using namespace eosio;

#define N(name) name##_n.value


struct record {
   uint64_t    primary;
   uint128_t   secondary;
   uint64_t    data;
   uint64_t primary_key() const { return primary; }
   uint128_t get_secondary() const { return secondary; }
   EOSLIB_SERIALIZE( record, (primary)(secondary)(data))
};

extern "C" {
    void apply( uint64_t receiver, uint64_t code, uint64_t action ) {
        name _self(receiver);

        if (N("sayhello") == action) {
            name code = "helloworld11"_n;
            uint64_t scope = "scope"_n.value;
            
            multi_index<"mytable4"_n,
                        record,
                        indexed_by< "bysecondary"_n,
                        const_mem_fun<record, uint128_t, &record::get_secondary> > > mytable( code, scope);
            
            auto itr = mytable.find(11);
            
            if (mytable.end() == itr) {
                mytable.emplace( _self, [&]( auto& row ) {
                    row.primary = 11;
                    row.secondary = 22;
                    row.data = 111;
                });
            } else {
                mytable.modify( itr, _self, [&]( auto& row ) {
                    row.data = 1111;
                    row.secondary = 44;
                });
            }
            
            itr = mytable.find(22);
            if (itr == mytable.end()) {
                mytable.emplace( _self, [&]( auto& row ) {
                    row.primary = 22;
                    row.secondary = 222;
                    row.data = 2222;
                });
            } else {
                print(itr->primary, " ", itr->secondary, "\n");
            }

            auto idx = mytable.get_index<"bysecondary"_n>();
            print(idx.begin() == idx.end());
            for ( auto itr = idx.begin(); itr != idx.end(); itr++ ) {
                print(itr->primary, " ", itr->secondary);
            }
        }
    }
}
'''

publish_cpp_contract('helloworld11', code)
print('done!')




done!


In [80]:
name = 'helloworld11'
try:
    r = eosapi.push_action(name, 'sayhello', b'hello,world', {name:'active'})
    print(r['processed']['action_traces'][0]['console'])
    print(r['processed']['elapsed'])
except Exception as e:
    print(e)

22 222
true
261
